<a href="https://colab.research.google.com/github/kunai-3txk/Compe_tonyobyo/blob/main/AMP_FeatureStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [8]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 56.8 MB/s eta 0:00:00


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import os
import seaborn as sns; sns.set()
import polars as pl

# Configuration

In [3]:
class config:
    MAX_PROTEINS = 20
    PROFILE_REPORT = True
    
class paths:
    SAMPLE_SUBMISSION = "/content/drive/MyDrive/AMP/example_test_files/sample_submission.csv"
    SUPPLEMENTAL_CLINICAL_DATA = "/content/drive/MyDrive/AMP/supplemental_clinical_data.csv"
    TRAIN_CLINICAL_DATA = "/content/drive/MyDrive/AMP/train_clinical_data.csv"
    TRAIN_PEPTIDES = "/content/drive/MyDrive/AMP/train_peptides.csv"
    TRAIN_PROTEINS = "/content/drive/MyDrive/AMP/train_proteins.csv"
    TEST_CLINICAL_DATA = "/content/drive/MyDrive/AMP/example_test_files/test.csv"
    TEST_PEPTIDES = "/content/drive/MyDrive/AMP/example_test_files/test_peptides.csv"
    TEST_PROTEINS = "/content/drive/MyDrive/AMP/example_test_files/test_proteins.csv"

# Function Definition

In [35]:
#統計量の作成
def make_statistic(df,group_col:str,agg_list:list):
    df_ret = df.groupby(group_col).agg([
        pl.mean(agg_list).suffix('_mean')
        ,pl.std(agg_list).suffix("_std")
        ,pl.median(agg_list).suffix("_med")
        #,pl.col(agg_list).max().suffix("_max") 
        #,pl.col(agg_list).min().suffix("_min") 
        #,pl.quantile(agg_list',0.25).suffix("_quan1")
        #,pl.quantile(agg_list',0.75).suffix("_quan3")
        ])
    return df_ret

# Load Data

In [5]:
train_clinical = pd.read_csv(paths.TRAIN_CLINICAL_DATA)
train_peptides = pd.read_csv(paths.TRAIN_PEPTIDES)
train_proteins = pd.read_csv(paths.TRAIN_PROTEINS)
supplemental_clinical = pd.read_csv(paths.SUPPLEMENTAL_CLINICAL_DATA)
#print('Correct label:Proteins shape: (232741, 5) Peptides shape: (981834, 6) Clinical shape: (2615, 8)')
print('Proteins shape:',train_proteins.shape)
print('Peptides shape:',train_peptides.shape)
print('Clinical shape:',train_clinical.shape)
print('supplemental_clinical shape:',supplemental_clinical.shape)

Proteins shape: (232741, 5)
Peptides shape: (981834, 6)
Clinical shape: (2615, 8)
supplemental_clinical shape: (2223, 8)


#Protain

In [48]:
train_proteins['UniProt'].value_counts()

P01024    1113
P05090    1113
P01011    1113
P01023    1113
Q92520    1113
          ... 
Q6UX71     661
P01780     654
Q562R1     616
Q99832     606
Q99829     489
Name: UniProt, Length: 227, dtype: int64

In [36]:
train_proteins_1 = make_statistic(pl.DataFrame(train_proteins),'visit_id',['NPX'])
train_proteins_1.head()

visit_id,NPX_mean,NPX_std,NPX_med
str,f64,f64,f64
"""27872_60""",3.1283e6,2.4573e7,125676.0
"""57416_12""",2.6081e6,2.0440e7,108875.0
"""47513_24""",2.6925e6,1.9648e7,113596.0
"""60443_0""",2.4679e6,1.8376e7,111823.0
"""55256_24""",3.1892e6,2.6121e7,123054.5


In [37]:
train_proteins_2 = make_statistic(pl.DataFrame(train_proteins),'UniProt',['NPX'])
train_proteins_2.head()

UniProt,NPX_mean,NPX_std,NPX_med
str,f64,f64,f64
"""P01859""",2.1470e6,990948.600782,2.05392e6
"""Q92520""",643569.644663,256993.316437,617515.0
"""P60174""",15290.254403,5378.119619,14906.2
"""P02763""",4.9391e6,2.2398e6,4.75978e6
"""P61278""",6086.982756,3588.640827,5460.78


#Peptide

In [49]:
train_peptides['UniProt'].value_counts()

P02787    51916
P02768    40900
P01024    35253
P02751    27370
P02649    23082
          ...  
Q6UX71      661
P01780      654
Q562R1      616
Q99832      606
Q99829      489
Name: UniProt, Length: 227, dtype: int64

In [40]:
train_peptides_1 = make_statistic(pl.DataFrame(train_peptides),'visit_id',['PeptideAbundance'])
train_peptides_1.head()

visit_id,PeptideAbundance_mean,PeptideAbundance_std,PeptideAbundance_med
str,f64,f64,f64
"""17201_36""",738587.288013,3.7071e6,65518.7
"""13618_12""",672089.679744,3.9313e6,64526.3
"""45662_12""",560386.504534,2.2051e6,75243.6
"""62723_84""",630870.538874,4.3069e6,65645.6
"""35231_48""",729022.384815,3.2331e6,81238.9


In [43]:
train_peptides_2 = make_statistic(pl.DataFrame(train_peptides),'UniProt',['PeptideAbundance'])
train_peptides_2.head()

UniProt,PeptideAbundance_mean,PeptideAbundance_std,PeptideAbundance_med
str,f64,f64,f64
"""P09104""",7050.218596,2721.082565,6700.845
"""P01033""",81673.321807,22340.620216,79785.9
"""P01877""",49611.635782,44742.754463,36338.5
"""P02452""",24747.40123,9347.599914,24249.8
"""P23142""",95865.319007,103851.314044,63137.8
"""P04406""",14891.194885,6351.602747,14104.1
"""P80748""",35114.835009,15510.986651,32947.5
"""P12109""",37473.205132,12885.264464,36264.25
"""P07333""",36381.649577,16715.408428,32679.0


#Clinical

In [54]:

#欠損値処理
train_clinical.dropna(subset=['updrs_1'],inplace=True)
train_clinical.dropna(subset=['updrs_2'],inplace=True)

#upd23dの置き換え



visit_id                               False
patient_id                             False
visit_month                            False
updrs_1                                False
updrs_2                                False
updrs_3                                 True
updrs_4                                 True
upd23b_clinical_state_on_medication     True
dtype: bool

#ProPep

In [23]:
train_ProPep = train_peptides.merge(train_proteins[['visit_id','UniProt','NPX']],on=['visit_id','UniProt'],how='left')
print(train_ProPep.shape)
train_ProPep.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [ ]:
train_ProPep_agg = train_ProPep.groupby('visit_id').mean()
train_all = train_clinical.merge(train_ProPep_agg[['PeptideAbundance','NPX']],on='visit_id',how='left')
print('shape',train_all.shape)
train_all.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance,NPX
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0,732430.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0,732430.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9,732430.0
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7,732430.0
...,...,...,...,...,...,...,...
645363,21126_36,36,21126,P10909,EILSVDC(UniMod_4)STNNPSQAK,251923.0,8338150.0
645364,21126_36,36,21126,P10909,EIQNAVNGVK,152618.0,8338150.0
645365,21126_36,36,21126,P10909,FM(UniMod_35)ETVAEK,86861.3,8338150.0
645366,21126_36,36,21126,P10909,FMETVAEK,1581860.0,8338150.0


#Train All

In [ ]:
df_train = train_all[['visit_id','visit_month','updrs_1','updrs_2','updrs_3','updrs_4','PeptideAbundance','NPX']]
df_train.head()
#df_train.info()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication,PeptideAbundance,NPX
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,748153.907014,2.028789e+07
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,685218.599872,1.843914e+07
3,55_9,55,9,8.0,9.0,30.0,0.0,On,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,On,740535.423563,2.010976e+07
...,...,...,...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0,Off,NaN,NaN
2611,65043_54,65043,54,4.0,8.0,11.0,1.0,Off,NaN,NaN
2612,65043_60,65043,60,6.0,6.0,16.0,1.0,Off,NaN,NaN
2613,65043_72,65043,72,3.0,9.0,14.0,1.0,Off,NaN,NaN


In [ ]:
#1行だけ欠損を処理
df_train=df_train.dropna(subset=['updrs_1'])

X = df_train.drop(['visit_id','updrs_1'],axis=1).values
y = df_train['updrs_1'].values
train_column_list = df_train.drop(['visit_id','updrs_1'],axis=1).columns

#X = df_train.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1).values
#y = df_train['updrs_{0}'.format(i + 1)].values